## Julia Basics

In [ ]:
a = 2         # Integer
b = 2.        # double or Floating point
c = 1//3      # rational number
d = 1 + 3im   # complex number
println(a, ", ", b, ", ", c, ", ", d)

In [ ]:
v = [1, 5, 7]

In [ ]:
isa(v, Vector)

Vectors are different from matrices and different from scalars

In [1]:
b = randn(3)  # vector of normal random numbers
A = [1 2 3;
     2 4 5;
     3 5 6]

3×3 Array{Int64,2}:
 1  2  3
 2  4  5
 3  5  6

In [ ]:
isa(A, Matrix)

In [ ]:
size(A)

In [ ]:
res = b.'*A*b   # this is NOT dotproduct

In [ ]:
typeof(res)     # Vector

In [ ]:
res = dot(b, A*b)

## Familiar syntax to Matlab

In [ ]:
det(A)                   # determinant
vals, vectors = eig(A)   # eigenvalues
A^2                      # matrix to power
A.^4                     # matrix to power elementwise

In [ ]:
u,s,v = svd(A)           # singular value decomposition
s

In [ ]:
norm(A)                  # matrix norm
A\b                      # solve linear system Ax = b

In [ ]:
?norm

# matrix manipulations

In [ ]:
A[:,[1,2]]

In [ ]:
zeros(3,3)

In [2]:
A + 3I + 3eye(3) + 3speye(3)

3×3 Array{Float64,2}:
 10.0   2.0   3.0
  2.0  13.0   5.0
  3.0   5.0  15.0

In [ ]:
diagm([1, 4, 5])

In [ ]:
vcat(A, [5 5 5])

In [ ]:
hcat(A, [5 5 5].')

Matrices have column major ordering

In [ ]:
a = 1:4

In [ ]:
b = collect(a)

In [ ]:
c = reshape(b,2,2)

In [ ]:
c[1,2] == c[3] == 3

Julia specializes functions depending on input types through multiple dispatch 

In [ ]:
d = diagm([2,3,4])

In [ ]:
isa(d, Matrix)

In [ ]:
A + d

In [ ]:
D = Diagonal([2,3,4])

Diagonal is a different type!

In [ ]:
isa(D, Matrix)            

But it can still be used as if it were a normal dense matrix

In [ ]:
A + D

In [ ]:
methods(+);

In [ ]:
@which A + D

## Loops

In [ ]:
A = randn(3,3)

for loops as we are used to them

In [ ]:
counts = 0
for row = 1:size(A,1)
  for col = 1:size(A,2)
    if A[row,col] ≤ 0
      counts += 1
    else
      nothing
    end
  end
end
counts

In [ ]:
for row = 1:size(A,1), col = 1:size(A,2)
  if A[row,col] ≤ 0
    counts += 1
  else
    nothing
  end
end
counts

better looking, and more efficient way


In [ ]:
for idx in eachindex(A)
  counts += A[idx] ≤ 0 ? 1 : 0
end
counts

can also iterate only over elements

In [ ]:
for elem in A
  counts += elem ≤ 0 ? 1 : 0
end
counts

works for any "iterable" type

In [ ]:
for elem in A
  counts += elem ≤ 0
end
println(counts)

isa(true, Integer)

In [ ]:
filter(x->x ≤ zero(x), A)

In [ ]:
counts = filter(x->x ≤ zero(x), A) |> length     # chaining the output of filter to length

## while loop

In [ ]:
kmax = 6
k = 1

while k < kmax
  # do something
  # do not forget to increment k
  # string interpolation with "$"
  println("k: $k")
  k += 1
end

# Functions, just in time compilation and multiple dispatch

In [ ]:
f(x,y) = x + y

In [ ]:
f(1,1)

In [ ]:
f(x::Int, y::Int) = 2x+y

In [ ]:
f(1,1)  # dispatches to most specialized method

In [ ]:
@time f(1,1)
@time f(1,1)
@time f(1,1.0)
@time f(1,1.0)

Julia, like many languages, compiles to LLVM and we can inspect the generated code (LLVM is a type of portable assembly language).

In [ ]:
@code_llvm f(1,1)

In [ ]:
a = 2.
b = √a       # \sqrt TAB a

In [ ]:
a = -1.
b = √a

In [ ]:
a = complex(a)
b = sqrt(a)

In [ ]:
function mysqrt(x::Real)
  try
    return √x
  catch err
    return √Complex128(x)
  finally
    println("mysqrt: I have finished")
  end
end

In [ ]:
mysqrt(-1)

## Type stability
Type stability is the idea that there is only 1 possible type which can be outputted from a method.
For example, the reasonable type to output from +(::Int,::Int) is an Int.
Julia is fast because it can specialize via multiple dispatch to type stable methods.

In [ ]:
@code_warntype sqrt(1.)

In [ ]:
@code_warntype mysqrt(1.)

# sets and dictionaries and tuples

In [ ]:
a = (1,2.0,3) |> typeof

In [ ]:
b = [1,2,3] |> typeof

Tuples cannot be changed

In [ ]:
a[1] = 2

In [ ]:
mycol = Set{Int}()
push!(mycol, (1,2,3,1,2,3)...)
push!(mycol, 1, 2, 3, 4, 1, 2)

In [ ]:
col1 = mycol
col2 = copy(mycol)   # pass by reference is default so we need to copy 

In [ ]:
pop!(col2, 1)        # remove 1 from the set

In [ ]:
res = col1 ∪ col2    # union() \cup

In [ ]:
res = col1 ∩ col2    # intersect() \cap

In [ ]:
res = setdiff(col1,col2)

In [ ]:
1 ∈ res          # in()

In [ ]:
res ∋ 1

In [ ]:
1 ∉ res

In [ ]:
dict = Dict("a" => 1, "b" => 2, "c" => 3)

In [ ]:
dict["b"]

In [ ]:
for (key, value) in dict
    println(key, " ==> ", value)
end

# Custom types

In [ ]:
type MyType
  α::Float64
end

In [ ]:
m = MyType(5)

In [ ]:
m.α = 3.

In [ ]:
m

In [ ]:
fieldnames(m)

In [ ]:
# Some not-trivial constructor
function (::Type{MyType})(x::Vector)
  MyType(sum(x))
end

In [ ]:
# Function call-overload: we are making MyType a callable object (i.e., a functor)
function (m::MyType)(x)
  return m.α * x
end

In [ ]:
m(5)

In [ ]:
import Base.+
+(m1::MyType, m2::MyType) = MyType(m1.α+m2.α)

In [ ]:
m2 = MyType([1,2,3])
mres = m+m2

# Function of functions
Function is a type and can be assigned to variables

In [ ]:
g = (m,x)->m(x)

In [ ]:
g(sqrt, 2)

In [ ]:
isa(g, Function)

Our custom type is also callable

In [ ]:
g(m, 3)

In [ ]:
g(m, randn(6))